# Nballs Generation

**Please read the [Readme](https://github.com/ghanem-mhd/N-Balls-Visualization) for more information.**

In [2]:
from ipywidgets import Layout
import ipywidgets as widgets
import os
from IPython.display import clear_output

SAMPLE = None

project_path        = os.path.dirname(os.getcwd())
data_path           = project_path + '/data'
custom_input        = data_path + "/custom_input.txt"
word2vec            = data_path + "/glove.txt"

def generate_nballs(glove_file_path, output_path, sample, input_file_path):
    generate_balls_string_command = "../main.py --generate_nballs --w2v %s --output %s --sample %s --input %s"
    command = generate_balls_string_command%(glove_file_path, output_path, sample, input_file_path)     
    %run $command
    
def write_data_to_file(file_path, lines):
    with open(file_path, 'w') as file:
        for line in lines:
            file.write(line)
            if not '\n' in line:
                file.write("\n")

# UI Widgets
input_examples         = widgets.Combobox( placeholder='Choose Example', description='Input:', options=['Cities','Fruits','Custom'], layout=Layout(height='auto', width='50%'))
check_input_button     = widgets.Button(description='Check', button_style='success')
words_input            = widgets.Textarea(value='',description='Words', placeholder='keywords comma separated',  layout=Layout(height='auto', width='50%'))
word2vec_path_input    = widgets.Text(placeholder='word2vec file path', value = '', description='word2vec', layout=Layout(height='auto', width='50%'))
generate_balls_button  = widgets.Button(description='Generate Balls', button_style='success')
buttons_box            = widgets.Box([word2vec_path_input,generate_balls_button])
custom_box             = widgets.Box([words_input,check_input_button])

# UI Widget functions
def on_input_example_choose(change):
    global SAMPLE
    if change['type'] == 'change' and change['name'] == 'value':
        choosen_values = change['new']
        input_examples.disabled = True
        SAMPLE = choosen_values
        if (choosen_values == 'Custom'):
            display(custom_box)
            SAMPLE = None
        else:
            display(buttons_box)

def on_check_button_clicked(button):
    parsed_tokens = [x.strip() for x in str(words_input.value).split(',')]
    words = [x for x in parsed_tokens if x]
    if len(words) > 0:
        words_input.disabled = True
        check_input_button.disabled = True
        write_data_to_file(custom_input,words)
        display(buttons_box)
    else:
        print("Can't parse your input! Try Again!")
        
def check_input_for_nballs():
    if os.path.exists(children_file_path) and os.path.exists(cat_code_file_path):
        display(generate_nballs_button)
        
def on_generate_ball_button_clicked(button):
    word2vec_path_input.disabled = True
    generate_balls_button.disabled = True
    if word2vec_path_input.value:
        generate_nballs(word2vec_path_input.value, data_path, SAMPLE, custom_input)
    else:
        print("Provide a path for word2Vec! Try again!")
    
# Set Widget listeners
input_examples.observe(on_input_example_choose)
check_input_button.on_click(on_check_button_clicked)
generate_balls_button.on_click(on_generate_ball_button_clicked)

# Start UI nteraction
display(input_examples)

Combobox(value='', description='Input:', layout=Layout(height='auto', width='50%'), options=('Cities', 'Fruits…

Box(children=(Textarea(value='', description='Words', layout=Layout(height='auto', width='50%'), placeholder='…

Box(children=(Text(value='', description='word2vec', layout=Layout(height='auto', width='50%'), placeholder='w…

Start generating tree files
32 words paths found
7 Leaves found
24 Subtree found
Finish generating tree files
Start generating balls
['entity.n.01']
*** entity.n.01
finished training of all families

loading balls....
[167]
totally 31  balls are loaded
updating first level children...
checking whether the tree structure is perfectly encoded in nball embeddings...

loading balls....
[167]
totally 69  balls are loaded
failed families with P []
failed families with DC []
the tree structure is perfectly encoded in nball embeddings.

generating nball embedding file...

Finish generating balls successfully
